<a href="https://colab.research.google.com/github/shubhigupta991/Reddit-Flair-Detection/blob/main/scripts/WebScrapping%20and%20PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting the data from reddit

We are collecting the data with the help of PRAW

PRAW stands for Python Reddit API Wrapper.

In [1]:
!pip install praw

     |████████████████████████████████| 153kB 10.9MB/s 
     |████████████████████████████████| 204kB 19.9MB/s 


In [2]:
import praw
import pandas as pd
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
import datetime as dt
nltk.download('all')
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [3]:
reddit = praw.Reddit(client_id='KqCyLYQgMNwp4w', client_secret='cA9UCAPiVadgs4FTsnZ3RqJUR0hROw', user_agent='Flair-Detector', 
                     username='shubhigupta09', password='flair123')

In [4]:
subreddit = reddit.subreddit('india')

In [5]:
labels = {"title":[], "id":[], "score":[], "url":[], "created": [], "num_of_comments": [], "body":[], "author":[], "comments":[], "flair":[]}
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", 
          "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA","Coronavirus"]

In [6]:
for flair in flairs:
  
  get_subreddits = subreddit.search(flair, limit=100)
  
  for each_post in get_subreddits:
    
    labels["flair"].append(flair)
    labels["title"].append(each_post.title)
    labels["score"].append(each_post.score)
    labels["id"].append(each_post.id)
    labels["url"].append(each_post.url)
    labels["num_of_comments"].append(each_post.num_comments)
    labels["created"].append(each_post.created)
    labels["body"].append(each_post.selftext)
    labels["author"].append(each_post.author)
    
    each_post.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in each_post.comments:
      comment = comment + ' ' + top_level_comment.body
    labels["comments"].append(comment)

In [7]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [9]:
data = pd.DataFrame(labels)
time =data["created"].apply(get_date)
data =data.assign(timestamp = time)
del data['created']
data.to_csv('reddit-india-data.csv', index=False)

In [10]:
data=pd.read_csv('reddit-india-data.csv')
data.head()

,title,id,score,url,num_of_comments,body,author,comments,flair,timestamp
0,How to approach a girl?,k0qt2r,3,https://www.reddit.com/r/india/comments/k0qt2r...,15,2 years back I was working in a startup compan...,covidmanbun,Stop watching Indian movies. \nStop stalking...,AskIndia,2020-11-25 19:50:12
1,Where is gelatine available ?,jk9zlt,0,https://www.reddit.com/r/india/comments/jk9zlt...,4,I wish to buy gelatine and am looking for the ...,csstudentG,It's available in your regular kirana stores....,AskIndia,2020-10-29 21:26:38
2,Trevor Noah's jokes during Indo-Pak tensions a...,gv9lmh,38,https://www.reddit.com/r/india/comments/gv9lmh...,29,I don't really watch much of Trevor Noah's lat...,CommYouNitty,It's mostly because we as a country don't rea...,AskIndia,2020-06-02 23:42:24
3,Need feedback for Insurance Policy that I took...,1s57oi,1,https://www.reddit.com/r/india/comments/1s57oi...,1,**Re-posting here because of lack of activity ...,dhavalcoholic,"Dear Policy Holder(Dhavalcoholic),\n \nWe req...",AskIndia,2013-12-05 14:30:23
4,"Buying used BS4 Scooty after April 1, Is there...",hpqq5o,12,https://www.reddit.com/r/india/comments/hpqq5o...,7,"I don't know if it's right place to ask, but ""...",akza07,Scammers will keep scamming. \n\nIt applies t...,AskIndia,2020-07-12 15:58:15


In [11]:
def string(value):
    return str(value)

In [12]:
data['title'] = string(data['title'])
data['body'] = string( data['body'])
data['comments'] = string(data['comments'])

In [13]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopWords = set(stopwords.words('english'))
def text_cleaning(text):
   
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = replace_by_space.sub(' ', text)
    text = bad_symbols.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopWords)
    return text

In [16]:
data['title'] = data['title'].apply(text_cleaning)
data['body'] = data['body'].apply(text_cleaning)
data['comments'] = data['comments'].apply(text_cleaning)

In [17]:
combined_features = data["title"] + data["comments"] + data["url"] + data["body"]
data = data.assign(combined_features = combined_features)

In [18]:
data.to_csv('data.csv')

In [19]:
pd.read_csv('data.csv')

,Unnamed: 0,title,id,score,url,num_of_comments,body,author,comments,flair,timestamp,combined_features
0,0,0 approach girl1 gelatine available 2 trevor n...,k0qt2r,3,https://www.reddit.com/r/india/comments/k0qt2r...,15,0 2 years back working startup compan1 wish bu...,covidmanbun,0 stop watching indian movies nstop stalking1 ...,AskIndia,2020-11-25 19:50:12,0 approach girl1 gelatine available 2 trevor n...
1,1,0 approach girl1 gelatine available 2 trevor n...,jk9zlt,0,https://www.reddit.com/r/india/comments/jk9zlt...,4,0 2 years back working startup compan1 wish bu...,csstudentG,0 stop watching indian movies nstop stalking1 ...,AskIndia,2020-10-29 21:26:38,0 approach girl1 gelatine available 2 trevor n...
2,2,0 approach girl1 gelatine available 2 trevor n...,gv9lmh,38,https://www.reddit.com/r/india/comments/gv9lmh...,29,0 2 years back working startup compan1 wish bu...,CommYouNitty,0 stop watching indian movies nstop stalking1 ...,AskIndia,2020-06-02 23:42:24,0 approach girl1 gelatine available 2 trevor n...
3,3,0 approach girl1 gelatine available 2 trevor n...,1s57oi,1,https://www.reddit.com/r/india/comments/1s57oi...,1,0 2 years back working startup compan1 wish bu...,dhavalcoholic,0 stop watching indian movies nstop stalking1 ...,AskIndia,2013-12-05 14:30:23,0 approach girl1 gelatine available 2 trevor n...
4,4,0 approach girl1 gelatine available 2 trevor n...,hpqq5o,12,https://www.reddit.com/r/india/comments/hpqq5o...,7,0 2 years back working startup compan1 wish bu...,akza07,0 stop watching indian movies nstop stalking1 ...,AskIndia,2020-07-12 15:58:15,0 approach girl1 gelatine available 2 trevor n...
...,...,...,...,...,...,...,...,...,...,...,...,...
1211,1211,0 approach girl1 gelatine available 2 trevor n...,j002jl,323,https://thewire.in/politics/bjp-bihar-election...,43,0 2 years back working startup compan1 wish bu...,mubukugrappa,0 stop watching indian movies nstop stalking1 ...,Coronavirus,2020-09-26 13:23:26,0 approach girl1 gelatine available 2 trevor n...
1212,1212,0 approach girl1 gelatine available 2 trevor n...,j0wucw,0,https://www.reddit.com/r/india/comments/j0wucw...,0,0 2 years back working startup compan1 wish bu...,pizzapuff93,0 stop watching indian movies nstop stalking1 ...,Coronavirus,2020-09-28 03:08:39,0 approach girl1 gelatine available 2 trevor n...
1213,1213,0 approach girl1 gelatine available 2 trevor n...,izx7aw,5,https://www.reddit.com/r/india/comments/izx7aw...,1,0 2 years back working startup compan1 wish bu...,spddgr8,0 stop watching indian movies nstop stalking1 ...,Coronavirus,2020-09-26 10:12:50,0 approach girl1 gelatine available 2 trevor n...
1214,1214,0 approach girl1 gelatine available 2 trevor n...,ix6pl0,1020,https://www.reddit.com/gallery/ix6pl0,153,0 2 years back working startup compan1 wish bu...,IndianPuppy,0 stop watching indian movies nstop stalking1 ...,Coronavirus,2020-09-22 02:55:01,0 approach girl1 gelatine available 2 trevor n...
